# Install and import libraries

In [ ]:
!pip install beautifulsoup4
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 911kB 6.4MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease

In [ ]:
import pandas as pd
import numpy as np

# Create the web driver
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: use options instead of chrome_options


# Load the site

In [ ]:
url = "https://ph.investing.com/equities/philippines"
driver.get(url)
driver.implicitly_wait(10)

# Show all stocks
filter = driver.find_element_by_css_selector("#stocksFilter")
filter.find_element_by_css_selector("option#all").click()

stock_links = []

try:
  links_selector = "#cross_rate_markets_stocks_1 tbody tr td:nth-child(2) a"
  anchors = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, links_selector)))

  for anchor in anchors:
    stock_links.append({
        "name": anchor.get_attribute("title"),
        "url": anchor.get_attribute("href") + "-historical-data"
    })

  print("Retrieved", len(stock_links), "URLs")
except TimeoutException:
  print("Element not properly loaded")

driver.close()

Retrieved 303 URLs


In [ ]:
def get_historical_data(stocks):
  t_driver = webdriver.Chrome('chromedriver', options=chrome_options)
  historical_data = []

  for stock in stocks:
    print("Start scraping", stock["name"])

    t_driver.get(stock["url"])
    t_driver.implicitly_wait(10)
    date_widget = t_driver.find_element_by_id("widgetField")
    t_driver.execute_script("arguments[0].click();", date_widget)

    stock_code = t_driver.find_element_by_tag_name("h2").get_attribute("innerHTML")
    stock_code = stock_code.split(" ")[0]

    start_date = t_driver.find_element_by_id("startDate")
    start_date.clear()
    start_date.send_keys("08/08/1927")

    apply_widget = t_driver.find_element_by_id("applyBtn")
    t_driver.execute_script("arguments[0].click();", apply_widget)

    data_rows_selector = "#curr_table tbody"
    data_rows = WebDriverWait(t_driver, 20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, data_rows_selector)))

    stock_data = []

    for data_row in data_rows:
      temp = []
      for data_col in data_row.find_elements_by_tag_name("td"):
        temp.append(data_col.get_attribute("innerHTML"))
      temp = [stock["name"], stock_code] + temp
      stock_data.append(temp)
      print(temp)

    historical_data += stock_data
    print("Finish scraping", stock["name"])

  t_driver.close()
  return historical_data

In [ ]:
columns = ["Stock Name", "Code", "Date", "Price", "Open", "High", "Low", "Volume", "Change%"]

# all_data = get_historical_data(stock_links[0:50])
# pd_data = pd.DataFrame(all_data, columns=columns)
# pd_data.to_csv("/content/drive/MyDrive/Colabs/Stock/stock1.csv", index=False)

In [ ]:
all_data = get_historical_data(stock_links[51:100])
pd_data = pd.DataFrame(all_data, columns=columns)
pd_data.to_csv("/content/drive/MyDrive/Colabs/Stock/stock2.csv", index=False)

Streaming output truncated to the last 5000 lines.
['Alliance Select Foods International Inc', 'FOOD', 'Mar 05, 2018', '0.560', '0.560', '0.570', '0.550', '219.00K', '0.00%']
['Alliance Select Foods International Inc', 'FOOD', 'Mar 02, 2018', '0.560', '0.560', '0.580', '0.560', '190.00K', '-3.45%']
['Alliance Select Foods International Inc', 'FOOD', 'Mar 01, 2018', '0.580', '0.560', '0.580', '0.550', '519.00K', '0.00%']
['Alliance Select Foods International Inc', 'FOOD', 'Feb 28, 2018', '0.580', '0.570', '0.580', '0.560', '353.00K', '1.75%']
['Alliance Select Foods International Inc', 'FOOD', 'Feb 27, 2018', '0.570', '0.570', '0.580', '0.560', '1.06M', '-1.72%']
['Alliance Select Foods International Inc', 'FOOD', 'Feb 26, 2018', '0.580', '0.580', '0.590', '0.570', '286.00K', '0.00%']
['Alliance Select Foods International Inc', 'FOOD', 'Feb 23, 2018', '0.580', '0.580', '0.580', '0.570', '605.00K', '-1.69%']
['Alliance Select Foods International Inc', 'FOOD', 'Feb 22, 2018', '0.590', '0.

In [ ]:
all_data = get_historical_data(stock_links[101:150])
pd_data = pd.DataFrame(all_data, columns=columns)
pd_data.to_csv("/content/drive/MyDrive/Colabs/Stock/stock3.csv", index=False)

In [ ]:
all_data = get_historical_data(stock_links[151:200])
pd_data = pd.DataFrame(all_data, columns=columns)
pd_data.to_csv("/content/drive/MyDrive/Colabs/Stock/stock4.csv", index=False)

In [ ]:
all_data = get_historical_data(stock_links[201:250])
pd_data = pd.DataFrame(all_data, columns=columns)
pd_data.to_csv("/content/drive/MyDrive/Colabs/Stock/stock5.csv", index=False)

In [ ]:
all_data = get_historical_data(stock_links[251:])
pd_data = pd.DataFrame(all_data, columns=columns)
pd_data.to_csv("/content/drive/MyDrive/Colabs/Stock/stock6.csv", index=False)